- We want to highlight 10 wines to increase our sales. Which ones should we choose and why?
- We have a limited marketing budget for this year. Which country should we prioritise and why?
- We would like to give awards to the best wineries. Come up with 3 relevant ones. Which wineries should we choose and why?
- We detected that a big cluster of customers likes a specific combination of tastes. We identified a few keywords that match these tastes: _coffee_, _toast_, _green apple_, _cream_, and _citrus_ (note that these keywords are case sensitive ⚠️). We would like you to find all the wines that are related to these keywords. Check that **at least 10 users confirm those keywords**, to ensure the accuracy of the selection. Additionally, identify an appropriate group name for this cluster.
- We would like to select wines that are easy to find all over the world. **Find the top 3 most common `grape`s all over the world** and **for each grape, give us the the 5 best rated wines**.
- We would like to create a country leaderboard. Come up with a visual that shows the **average wine rating for each `country`**. Do the same for the `vintages`.
- One of our VIP clients likes _Cabernet Sauvignon_ and would like our top 5 recommendations. Which wines would you recommend to him?

In [2]:
import sqlite3


import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


In [3]:
connexion = sqlite3.connect("../data/vivino.db")
cursor = connexion.cursor()

In [4]:
# - We want to highlight 10 wines to increase our sales. 
# Which ones should we choose and why?

cursor.execute("""
               select   wines.name,
                        wines.ratings_average, 
                        sum(wines.ratings_count) as total_rating, 
                        countries.users_count
               from countries 
                    join regions on regions.country_code = countries.code
                    join wines on wines.region_id = regions.id
                group by wines.name
                order by wines.ratings_average desc,  total_rating desc
                limit 10
               """)

top_wines = cursor.fetchall()

# Display the top 10 wines
for wine in top_wines:
    print(f"Wine: {wine[0]}, Average Rating: {wine[1]}, Total Ratings: {wine[2]}")


# Those are the most high ranked and popular wines 



Wine: Amarone della Valpolicella Classico Riserva, Average Rating: 4.8, Total Ratings: 587
Wine: Fratini Bolgheri Superiore, Average Rating: 4.8, Total Ratings: 153
Wine: Mágico, Average Rating: 4.8, Total Ratings: 146
Wine: Cristal Rosé Vinothèque, Average Rating: 4.8, Total Ratings: 88
Wine: Batard-Montrachet Grand Cru, Average Rating: 4.8, Total Ratings: 34
Wine: Unico, Average Rating: 4.7, Total Ratings: 45140
Wine: Sauternes, Average Rating: 4.7, Total Ratings: 45064
Wine: Special Selection Cabernet Sauvignon, Average Rating: 4.7, Total Ratings: 41236
Wine: Grange, Average Rating: 4.7, Total Ratings: 24356
Wine: Unico Reserva Especial Edición, Average Rating: 4.7, Total Ratings: 13025


In [5]:
# - We have a limited marketing budget for this year. Which country should we prioritise and why?
cursor.execute("""
               select countries.name, countries.users_count
                from countries
                order by countries.users_count desc
                limit 5
               """)

countries_top = cursor.fetchall()

for country in countries_top:
    print(f"Country: {country[0]}, Nb_users: {country[1]}")

# We should prioritize the countries with the most clients already, beceause the probability concerning the cost/ratio is better 


Country: États-Unis, Nb_users: 12273684
Country: France, Nb_users: 5973301
Country: Italie, Nb_users: 4270717
Country: Allemagne, Nb_users: 2549989
Country: Espagne, Nb_users: 2264396


In [6]:
# - We would like to give awards to the best wineries. 
# Come up with 3 relevant ones. Which wineries should we choose and why?
import csv
import sys

query = """
                select   wines.winery_id,
                        avg(wines.ratings_average) as average_rating,
                        count(wines.id) as number_of_wines
                from wines
                group by wines.winery_id
                order BY average_rating desc
               """
               
try : 
    cursor.execute(query)
    top_wineries = cursor.fetchall()
except (sqlite3.connector.Error, sqlite3.DatabaseError) as e:
    print(f"Database error occurred: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")
    

# Continue only if there are rows returned.
if top_wineries:
    # Write result to file.
    headers = [col[0] for col in cursor.description] # get headers
    top_wineries.insert(0, tuple(headers))
    with open("../data/test.csv", 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csvwriter.writerows(top_wineries)
else:
    sys.exit("No rows found for query: {}".format(query))

# # Display the top 10 wines
# for winery in top_wineries:
#     print(f"Winery: {winery[0]}, Average rating of wines: {winery[1]:.2f}, Number of wines : {winery[2]}")

In [12]:
# - We detected that a big cluster of customers likes a specific combination of tastes. 
# We identified a few keywords that match these tastes: 
# _coffee_, _toast_, _green apple_, _cream_, and _citrus_ (note that these keywords are case sensitive ⚠️). 
# We would like you to find all the wines that are related to these keywords. 
# Check that **at least 10 users confirm those keywords**, to ensure the accuracy of the selection. 
# Additionally, identify an appropriate group name for this cluster.

# All tastes

query = """
    
    select w.name, group_concat(distinct k.name) as keywords,count(distinct kw.keyword_id) as nb_keywords, c.name, c.users_count
    from keywords_wine kw
        join keywords k on kw.keyword_id = k.id
        join wines w on kw.wine_id = w.id
        join regions r on w.region_id = r.id
        join countries c on r.country_code = c.code
    WHERE 
        k.name IN ('coffee', 'toast', 'green apple', 'cream', 'citrus') and 
        c.users_count >= 10
    GROUP BY 
        w.name, w.id
    HAVING 
        COUNT(DISTINCT k.name) = 5;
    """
            

cursor.execute(query)
favorites_taste = cursor.fetchall()

    
favorites_taste

[("'Giulio Ferrari' Riserva del Fondatore",
  'citrus,coffee,cream,green apple,toast',
  5,
  'Italie',
  4270717),
 ('5 Puttonyos Aszú (Blue Label)',
  'citrus,coffee,cream,green apple,toast',
  5,
  'Hongrie',
  102235),
 ('Aconcagua Valley',
  'citrus,coffee,cream,green apple,toast',
  5,
  'Chili',
  326757),
 ("Aile d'Argent Blanc Bordeaux",
  'citrus,coffee,cream,green apple,toast',
  5,
  'France',
  5973301),
 ('Alexander Valley Cabernet Sauvignon',
  'citrus,coffee,cream,green apple,toast',
  5,
  'États-Unis',
  12273684),
 ('Amarone della Valpolicella Classico',
  'citrus,coffee,cream,green apple,toast',
  5,
  'Italie',
  4270717),
 ('Amarone della Valpolicella Classico',
  'citrus,coffee,cream,green apple,toast',
  5,
  'Italie',
  4270717),
 ('Amour de Deutz Millesimé Rosé Brut Champagne',
  'citrus,coffee,cream,green apple,toast',
  5,
  'France',
  5973301),
 ('Amour de Deutz Millésimé Brut Champagne',
  'citrus,coffee,cream,green apple,toast',
  5,
  'France',
  597330

In [8]:
# - We would like to select wines that are easy to find all over the world. 
# Find the top 3 most common `grape`s all over the world** and **for each grape, give us the the 5 best rated wines.


In [9]:
# - We would like to create a country leaderboard. 
# Come up with a visual that shows the **average wine rating for each `country`**. 
# Do the same for the `vintages`.


In [10]:
# - One of our VIP clients likes _Cabernet Sauvignon_ and would like our top 5 recommendations. 
# Which wines would you recommend to him?